# STA 141B Assignment 1

Due __January 26, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. The total number of points is 10. 

__Exercise 1__

This exercise will review basic concepts of programming. Only use pure python code and no methods (like `str.find`) that are optimized in, e.g., `C`. Likewise, do not use any packages except those suitable for parallelization in part (c). 



__(a)__ Write a recursive function `seq_count(x, ...)` that returns length of the longest subsequence of identical elements in the sequence object `x`. Run: 
```
seq_count([[1], [1], [1], 1, 3, 3, 2, 2, 4, 0])
seq_count(('G', 'g', 'a', "a", "a", '''a''', 2, 's', 's'))
seq_count([3, 1, int(True), 1, 1, 1, 3, 3])
seq_count((1, 3, None, 3, 3, 1, 3, 3, 4, 0))
```

In [2]:
def seq_count(x, curr_count = 1, max_count = 1, curr_char = None):
    if not x:
        return max_count
    if x[0] == curr_char:
        curr_count += 1
    else:
        curr_count = 1
    max_count = max(curr_count, max_count)
    return seq_count(x[1:], curr_count, max_count, x[0])
    

In [3]:
seq_count([1, 3, 1, 1, 3, 3, 4, 4, 4])

3

In [8]:
# seq_count([[1], [1], [1], 1, 3, 3, 2, 2, 4, 0])
#seq_count(('G', 'g', 'a', "a", "a", '''a''', 2, 's', 's'))
#seq_count([3, 1, int(True), 1, 1, 1, 3, 3])
#seq_count((1, 3, None, 3, 3, 1, 3, 3, 4, 0))
seq_count((1, 3, 1, 1, 1, '1', 1, [3, 3, 3, 3], 3, 4, 0))

3

__(b)__ Write a function `pattern_count(x, pattern, ...)` that takes the two iterable objects `x` and `pattern` and returns the length of the longest subsequence of `pattern`. Run: 
```
pattern_count('CGGACTACTAGACT', 'ACT')
pattern_count((1, (1, 1, 1, 1), 2, 1, 1, 1), [1, 1])
pattern_count(['ab', 'ab', 'a', 'a', 'b'], ('ab',))
```

In [70]:
def pattern_count(sequence, pattern):
    # Initialize variables to keep track of current match and longest match
    current_count = 0
    max_count = 0

    # Iterate through the sequence
    for item in sequence:
        if item == pattern[current_count]:
            # If the current item matches the pattern, increment the count
            current_count += 1
            if current_count == len(pattern):
                # If the entire pattern is matched, update the max_count
                max_count = max(max_count, current_count)
                current_count = 0  # Reset current count for the next potential match
        else:
            # If there is a mismatch, reset the current count
            current_count = 0

    return max_count

# def pattern_count(x, pattern):
#     max_count = 0
#     for seq in x:
#         n = len(str(seq))
#         curr_counter = 0
#         if n < len(pattern):
#             continue
#         for i in range(n):
#             if seq[i] == pattern[0]:
#                 for j in range(1, len(pattern)):
#                     if pattern[j] != seq[i + j]:
#                         break
#                     curr_counter += 1
            

#         max_count = max(max_count, curr_counter)
#     return max_count
        

In [71]:
pattern_count([0, 1, 2, 1, 2, 3, 1, 2, 1, 2, 1, 2, 4, 1, 2], (1, 2)) #should be 3

2

In [72]:
pattern_count([], [2])

0

In [73]:
pattern_count(['ab', 'ab', 'a', 'a', 'b'], 'ab') # elements in pattern must be identical to elements in x! 

0

In [61]:
pattern_count((1, (1, 1, 1, 1), 2, 1, 1, 1), [1, 1])

TypeError: object of type 'int' has no len()

__(c)__ For a long string, write code that takes strings `x`, `pattern`, and an integer `n_splits`, and uses a suitable concurrency method to search for repeating patterns using `pattern_count` from (b). 
To this end, partition `x` into `n_splits` parts and search each of them individually. Make sure not to split where a pattern is present! Run: 
```
from random import choices, seed

seed(2024)
x = "".join(choices('01', k = 5_000))
pattern = "01"
n_splits = 50

# here is your code 
```

_Hint: You can use the fast `x.find(pattern)` to check your code._

In [ ]:
from random import choices, seed

seed(2024)
x = "".join(choices('01', k = 5_000))
pattern = "01"
n_splits = 50

# here is your code 

__Exercise 2__

In this exercise, we will generate (pseudo-)random numbers using the inversion and accept-reject method. In order to generate the random numbers you are only allowed draw from the Uniform distribution and use
```
from random import uniform
from scipy.special import binom
from numpy import sqrt, pi, exp, tan, cumsum
from scipy.stats import probplot
import pandas as pd
import matplotlib.pyplot as plt
```

_Inversion method: Let $F$ be a distribution function from which we want to draw. Define the quantile function $F^{-1}(u) = \inf\{x\colon F(x)\geq u, 0\leq u\leq 1\}$. Then, if $U\sim Unif[0,1]$, $F^{-1}(U)$ has distribution function $F$._

_Accept-reject: Let $f$ be a density function from which we want to draw and there exists a density $g$ from which we can draw (e.g., via the inversion method) and for which there exists a constant $c$ such that $f(x)\leq cg(x)$ for all $x$. The following algorithm generates a random variable $X$ with density function $f$._

1. Generate a random variable $X$ from density $g$
2. Generate a random variable $U\sim Unif[0,1]$ (independent from $X$)
3. If $Ucg(X) \leq f(X)$, return $X$, otherwise repeat 1.-3.

_The number of iterations needed to successfully generate $X$ is itself a random variable,
which is geometrically distributed with the success (acceptance) probability $p = P(Ucg(X)\leq f(X))$. Hence, the expected number of iterations is $1/p$. Some calculations show that $p = 1/c$._


__(a)__ Generate $10 000$ samples from $Bin(10, 0.4)$ using __(i)__ the inversion method directly and __(ii)__ using the inversion method to draw corresponding Bernoulli distributed samples. __(iii)__ Plot the resulting empirical distribution functions and add the theoretical distribution function in one figure. 

In [1]:
from random import uniform
from scipy.special import binom
from numpy import sqrt, pi, exp, tan, cumsum
from scipy.stats import probplot
import pandas as pd
import matplotlib.pyplot as plt

__(b)__ Generate $10000$ samples from the standard normal distribution using the accept-reject method with candidate density $g(x) = (\pi(1+x^2))^{-1}$ with distribution funciton $G(x) = \tan^{-1}(x)/\pi$ from the standard Cauchy distribution. To this end, __(i)__ determine (mathematically or via simulation) the value of $c\geq1$ closest to one so that $f(x)\leq cg(x)$ for all $x$. __(ii)__ Obtain $10000$ standard normal random variables using the accept-reject method, generating Cauchy distributed random variables using inversion method. 
__(iii)__ Compare estimated and theoretical acceptance probabilities. 
__(iv)__ Generate a QQ-plot of the generated sample. 

__Exercise 3__

The demographic makeup of regions can offer crucial insights into various socio-economic factors. For policymakers, understanding age distributions can be particularly useful, as it can provide direction for initiatives ranging from educational policy to elderly care. In this section, we will work with a dataset detailing the age distribution across United States counties, broken down into specific age bins.

The files `county_age_dist.csv`, `fips_state.csv` and `fips_county.csv` contain information about the age distribution of counties in selected brackets as well as names and [FIPS](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) codes and additional information. 

__(a, i)__ Merge all three data frames into one `pandas.DataFrame` object names `data` with appropriate column names. __(ii)__ Remove the `info` column. Standardize column names and entries to be capitalized according to spelling rules. Remove any preceding whitespace if present for any entries. Run: 
```
data.head(4)
```

In [92]:
import pandas as pd
county_age = pd.read_csv('county_age_dist.csv') #3220 lines
fips_county = pd.read_csv('fips_county.csv') #3199 lines
fips_state = pd.read_csv('fips_state.csv') #51 lines
print(county_age.head(), fips_county.head(), fips_state.head())






   fips   0-17  18-24  25-34  35-44  45-54  55-64  65-74  75-84   85+
0  1001  25941  11422  12315  13828  14000  12697   9594   5430  1945
1  1003  86587  37568  44133  46730  49675  52405  43252  23262  8854
2  1005  11057   6162   6603   5907   6490   6377   5255   2795  1074
3  1007   9671   5241   5788   5472   6707   5563   4270   2555   638
4  1009  25671  11360  12635  13570  14737  14123  12106   6560  2022     fips             name  info
0  01000          Alabama   NaN
1  01001   Autauga County   NaN
2  01003   Baldwin County   NaN
3  01005   Barbour County   NaN
4  01007      Bibb County   NaN       FIPS; STATE
0     01; ALABAMA
1      02; ALASKA
2     04; ARIZONA
3    05; ARKANSAS
4  06; CALIFORNIA


,fips,name,info
0,01000,Alabama,NaN
1,01001,Autauga County,NaN
2,01003,Baldwin County,NaN
3,01005,Barbour County,NaN
4,01007,Bibb County,NaN


In [93]:
print(county_age.columns, fips_county.columns, fips_state.columns)

Index(['fips', '0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65-74',
       '75-84', '85+'],
      dtype='object') Index(['fips', ' name', ' info'], dtype='object') Index(['FIPS; STATE'], dtype='object')


In [94]:
county_age['fips'] = county_age['fips'].astype(str).str.zfill(5)
data = pd.merge(county_age, fips_county, how='outer', on='fips')

In [98]:
fips_value_placeholder = data['fips']
data['fips'] = data['fips'].astype(str).str[:2]


In [103]:
fips_state[['fips', 'state']] = fips_state['FIPS; STATE'].str.split('; ', expand=True)
fips_state = fips_state.drop('FIPS; STATE', axis=1)
fips_state.head()

,fips,state
0,01,ALABAMA
1,02,ALASKA
2,04,ARIZONA
3,05,ARKANSAS
4,06,CALIFORNIA


In [104]:
data = pd.merge(data, fips_state, how = 'outer', on='fips')
data.head()

,fips,0-17,18-24,25-34,35-44,45-54,55-64,65-74,75-84,85+,name,info,state
0,01,25941.0,11422.0,12315.0,13828.0,14000.0,12697.0,9594.0,5430.0,1945.0,Autauga County,NaN,ALABAMA
1,01,86587.0,37568.0,44133.0,46730.0,49675.0,52405.0,43252.0,23262.0,8854.0,Baldwin County,NaN,ALABAMA
2,01,11057.0,6162.0,6603.0,5907.0,6490.0,6377.0,5255.0,2795.0,1074.0,Barbour County,NaN,ALABAMA
3,01,9671.0,5241.0,5788.0,5472.0,6707.0,5563.0,4270.0,2555.0,638.0,Bibb County,NaN,ALABAMA
4,01,25671.0,11360.0,12635.0,13570.0,14737.0,14123.0,12106.0,6560.0,2022.0,Blount County,NaN,ALABAMA


In [119]:
data['fips'] = fips_value_placeholder
data = data.drop('info', axis = 1)
data['state'] = data['state'].str.lower().str.capitalize()
data.rename(columns={'fips': 'FIPS'}, inplace=True)
data.head()

,FIPS,0-17,18-24,25-34,35-44,45-54,55-64,65-74,75-84,85+,name,state
0,01001,25941.0,11422.0,12315.0,13828.0,14000.0,12697.0,9594.0,5430.0,1945.0,Autauga County,Alabama
1,01003,86587.0,37568.0,44133.0,46730.0,49675.0,52405.0,43252.0,23262.0,8854.0,Baldwin County,Alabama
2,01005,11057.0,6162.0,6603.0,5907.0,6490.0,6377.0,5255.0,2795.0,1074.0,Barbour County,Alabama
3,01007,9671.0,5241.0,5788.0,5472.0,6707.0,5563.0,4270.0,2555.0,638.0,Bibb County,Alabama
4,01009,25671.0,11360.0,12635.0,13570.0,14737.0,14123.0,12106.0,6560.0,2022.0,Blount County,Alabama


__(b)__ For each county and state, compute the proportion of elderly `CPE` and `SPE` (65 and older) to the total population as well as the proportion of young people `CPY` and `SPY` (24 or younger). Add those values to the data frame. You may ignore all FIPS regions that are not in states. Run: 
```
data.head(4)
```

In [135]:
# data.iloc[:, 1:9] = data.iloc[:, 1:9].apply(pd.to_numeric, errors='coerce')
data['0-17']

0       25941.0
1       86587.0
2       11057.0
3        9671.0
4       25671.0
         ...   
3281    27016.0
3282     4724.0
3283    11353.0
3284    16068.0
3285    17375.0
Name: 0-17, Length: 3286, dtype: float64

In [137]:
data['CPY'] = (sum(data.iloc[:, 1:6]))/ (sum(data.iloc[:,1:9]))
data.head()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [125]:
data.iloc[1000:1004,:]

,FIPS,0-17,18-24,25-34,35-44,45-54,55-64,65-74,75-84,85+,name,state,CPY
1000,21015,254825.0,116058.0,134015.0,115104.0,117175.0,121444.0,80369.0,45953.0,21417.0,Sedgwick County,Kansas,NaN
1001,21017,13931.0,6183.0,6546.0,5392.0,5194.0,4655.0,2959.0,1777.0,679.0,Seward County,Kansas,NaN
1002,21019,83441.0,37898.0,43565.0,38611.0,42499.0,46549.0,32866.0,18560.0,9490.0,Shawnee County,Kansas,NaN
1003,21021,1042.0,393.0,552.0,474.0,541.0,691.0,480.0,392.0,176.0,Sheridan County,Kansas,NaN
